In [14]:
from sklearn import datasets
import numpy as np

iris = datasets.load_diabetes()

X = iris.data
y = iris.target

In [23]:
iris.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
model.score(X_test, y_test)

0.3675149756138415

In [27]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def predict():
    features = [
        float(request.args.get('age')),
        float(request.args.get('sex')),
        float(request.args.get('bmi')),
        float(request.args.get('blood_pressure')),
        float(request.args.get('s1')),
        float(request.args.get('s2')),
        float(request.args.get('s3')),
        float(request.args.get('s4')),
        float(request.args.get('s5')),
        float(request.args.get('s6'))
    ]
    
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    print(prediction)

    return jsonify({'prediction': str(prediction[0]), 'model': 'XGBoost'})


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Feb/2025 16:32:27] "GET /predict?age=50&sex=1&bmi=25.3&blood_pressure=85&s1=200&s2=150&s3=100&s4=4.5&s5=5.2&s6=90 HTTP/1.1" 200 -


[255.14595]


127.0.0.1 - - [03/Feb/2025 16:41:26] "GET /predict?age=50&sex=1&bmi=25.3&blood_pressure=85&s1=200&s2=150&s3=100&s4=4.5&s5=5.2&s6=90 HTTP/1.1" 200 -


[255.14595]


127.0.0.1 - - [03/Feb/2025 16:45:00] "GET /predict?age=50&sex=1&bmi=25.3&blood_pressure=85&s1=200&s2=150&s3=100&s4=4.5&s5=5.2&s6=90 HTTP/1.1" 200 -


[255.14595]


## Question 2 - Using ngrok

In [ ]:
import requests
from collections import Counter

urls = [
    "localhost:5000",
    "https://6707-89-30-65-3.ngrok-free.app",
    "https://d3f4-89-30-65-3.ngrok-free.app",
    # ""
]


def get_predictions(urls):
    predictions = []
    
    for url in urls:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
        data = response.json()
        if 'prediction' in data:
            predictions.append(data['prediction'])
            print(data)

    
    if predictions:
        counter = Counter(predictions)
        most_common = counter.most_common(1)[0][0]  # Get the most frequent string
        return most_common
    else:
        return None
    

get_predictions(urls)

## Q3 - Weighting system

In [ ]:
weights = {url: 1.0 for url in urls}

def update_weights(predictions, true_value):
    """ Update weights based on their prediction accuracy. """

    # TODO estimate distance from average and penalize those too far from average

    for url, prediction in predictions.items():
        if prediction == true_value:
            weights[url] = min(weights[url] + 0.1, 1.0)  # Increase weight
        else:
            weights[url] = max(weights[url] - 0.1, 0.0)  # Decrease weight

def get_weighted_prediction(predictions):
    """ Calculate the weighted prediction. """
    weighted_counts = Counter()
    for url, prediction in predictions.items():
        weighted_counts[prediction] += weights[url]
    
    most_common_weighted = weighted_counts.most_common(1)[0][0]  # Get the most frequent, weighted string
    return most_common_weighted